# Vamos a trabajar acerca de las situaciones de las estaciones con el fin de trabajar en el mapa que contenga los datos sobre las bicis disponibles y no

In [2]:

import json
from pyspark import SparkContext
sc = SparkContext()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at /tmp/ipykernel_5644/119456393.py:3 

 Es importante recordar que nuestra base de datos es un diccionario con dos claves:
 * _id
 * stations que es una lista de diccionarios en la que cada diccionario es de la forma:

"stations": [{"activate": 1, "name": "Puerta del Sol A", "reservations_count": 0, "light": 0, "total_bases": 30, "free_bases": 26, "number": "1a", "longitude": "-3.7018341", "no_available": 0, "address": "Puerta del Sol n\\u00ba 1", "latitude": "40.4172137", "dock_bikes": 3, "id": 1}, ...]

In [3]:

rdd_base = sc.textFile('SituacionesEstaciones12_2020.json')

def mapper(line):
    data = json.loads(line)
    _id = data['_id']
    st = data['stations']
    return _id,st

rdd_2 = rdd_base.map(mapper)
rdd_2.take(1)



[('2020-12-01T00:56:04.683755',
  [{'activate': 1,
    'name': 'Puerta del Sol A',
    'reservations_count': 0,
    'light': 0,
    'total_bases': 30,
    'free_bases': 26,
    'number': '1a',
    'longitude': '-3.7018341',
    'no_available': 0,
    'address': 'Puerta del Sol nº 1',
    'latitude': '40.4172137',
    'dock_bikes': 3,
    'id': 1},
   {'activate': 1,
    'name': 'Puerta del Sol B',
    'reservations_count': 0,
    'light': 2,
    'total_bases': 30,
    'free_bases': 18,
    'number': '1b',
    'longitude': '-3.701602938060457',
    'no_available': 0,
    'address': 'Puerta del Sol nº 1',
    'latitude': '40.41731271011562',
    'dock_bikes': 9,
    'id': 2},
   {'activate': 1,
    'name': 'Miguel Moya',
    'reservations_count': 0,
    'light': 0,
    'total_bases': 24,
    'free_bases': 19,
    'number': '2',
    'longitude': '-3.7058415',
    'no_available': 0,
    'address': 'Calle Miguel Moya nº 1',
    'latitude': '40.4205886',
    'dock_bikes': 3,
    'id': 3},
  

**Si solo quiero _id** que es información asociada a las fechas ya tengo la función correspondiente


In [3]:
# 
def mapper_id(line):
    data = json.loads(line)
    _id = data['_id']
    return _id

rdd_id = rdd_base.map(mapper_id)
print("Las fechas son de la siguiente forma", rdd_id.take(3))

#Para pasarlo a un formato horario manejable
from datetime import datetime
def fecha_util(line):

    return datetime.strptime(line, "%Y-%m-%dT%H:%M:%S.%f")

print("\nUn ejemplo de convertirlo a fecha Año-Mes-Dia,Hora,Minutos,Segundos es:",fecha_util('2020-12-01T00:56:04.683755'))

rdd_fechas = rdd_id.map(fecha_util)
rdd_fechas.take(3)

Las fechas son de la siguiente forma ['2020-12-01T00:56:04.683755', '2020-12-01T01:56:05.745624', '2020-12-01T02:56:07.043903']

Un ejemplo de convertirlo a fecha Año-Mes-Dia,Hora,Minutos,Segundos es: 2020-12-01 00:56:04.683755


[datetime.datetime(2020, 12, 1, 0, 56, 4, 683755),
 datetime.datetime(2020, 12, 1, 1, 56, 5, 745624),
 datetime.datetime(2020, 12, 1, 2, 56, 7, 43903)]

Ahora vamos a trabajar solo con las estaciones

In [13]:
def mapper_st(line):
    data = json.loads(line)
    st = data['stations']
    return st

rdd_stations = rdd_base.map(mapper_st)
type(rdd_stations.take(2))

df = spark.createDataFrame(rdd_stations)
#Pongamos que quiero acceder al primer elemento que es la estacion Puerta del solA
#rdd_stations2 = rdd_stations.map(lambda x: x[0] )
#rdd_stations2.take(3)


NameError: name 'spark' is not defined

Ahora trabajaremos con todo

In [5]:
from ast import literal_eval
rdd_total = rdd_base.map(mapper)
def f(row):
    info_dict = literal_eval(row)
    output = []
    for st in info_dict["stations"]:
        print(st)
        output.append({"time" : datetime.strptime(info_dict["_id"], "%Y-%m-%dT%H:%M:%S.%f"),"name": st["name"], "pos": (st["longitude"],st["latitude"]) , "dock_bikes": st["dock_bikes"] })
    return output

rdd_total = rdd_base.map(f)
type(rdd_total.take(1))




{'activate': 1, 'name': 'Puerta del Sol A', 'reservations_count': 0, 'light': 0, 'total_bases': 30, 'free_bases': 26, 'number': '1a', 'longitude': '-3.7018341', 'no_available': 0, 'address': 'Puerta del Sol nº 1', 'latitude': '40.4172137', 'dock_bikes': 3, 'id': 1}
23/05/12 00:04:00 ERROR Executor: Exception in task 0.0 in stage 2.0 (TID 2)/ 1]
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/david/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 830, in main
    process()
  File "/home/david/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 822, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/david/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/david/.local/lib/python3.10/site-packages/pyspark/rdd.py", l

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2) (10.0.2.15 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/david/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 830, in main
    process()
  File "/home/david/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 822, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/david/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/david/.local/lib/python3.10/site-packages/pyspark/rdd.py", line 2830, in takeUpToNumLeft
    yield next(iterator)
  File "

In [18]:
rdd_total_columna = rdd_total.map(lambda x: x[1]['name'])

rdd_total_columna.take(2)

aux =[]
for i in range(3):
    aux.append(rdd_total.map(lambda x: x[i]['name']))

aux[0]

{'activate': 1, 'name': 'Puerta del Sol A', 'reservations_count': 0, 'light': 0, 'total_bases': 30, 'free_bases': 26, 'number': '1a', 'longitude': '-3.7018341', 'no_available': 0, 'address': 'Puerta del Sol nº 1', 'latitude': '40.4172137', 'dock_bikes': 3, 'id': 1}
{'activate': 1, 'name': 'Puerta del Sol B', 'reservations_count': 0, 'light': 2, 'total_bases': 30, 'free_bases': 18, 'number': '1b', 'longitude': '-3.701602938060457', 'no_available': 0, 'address': 'Puerta del Sol nº 1', 'latitude': '40.41731271011562', 'dock_bikes': 9, 'id': 2}
{'activate': 1, 'name': 'Miguel Moya', 'reservations_count': 0, 'light': 0, 'total_bases': 24, 'free_bases': 19, 'number': '2', 'longitude': '-3.7058415', 'no_available': 0, 'address': 'Calle Miguel Moya nº 1', 'latitude': '40.4205886', 'dock_bikes': 3, 'id': 3}
{'activate': 1, 'name': 'Plaza Conde Suchil', 'reservations_count': 0, 'light': 1, 'total_bases': 18, 'free_bases': 5, 'number': '3', 'longitude': '-3.7069171', 'no_available': 0, 'address':

KeyboardInterrupt: 

In [ ]:
def columna(line):
    data = json.loads(line)
    
    return data
#Tomamos un solo elemento para que veamos el nombre de todas las estaciones





In [ ]:
rdd_estaciones.take(1)

NameError: name 'rdd_estaciones' is not defined

In [ ]:
t = {"_id": "2020-12-01T00:56:04.683755", "stations": [{"activate": 1, "name": "Puerta del Sol A", "reservations_count": 0, "light": 0, "total_bases": 30, "free_bases": 26, "number": "1a", "longitude": "-3.7018341", "no_available": 0, "address": "Puerta del Sol n\u00ba 1", "latitude": "40.4172137", "dock_bikes": 3, "id": 1}, {"activate": 1, "name": "Puerta del Sol B", "reservations_count": 0, "light": 2, "total_bases": 30, "free_bases": 18, "number": "1b", "longitude": "-3.701602938060457", "no_available": 0, "address": "Puerta del Sol n\u00ba 1", "latitude": "40.41731271011562", "dock_bikes": 9, "id": 2}, {"activate": 1, "name": "Miguel Moya", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 19, "number": "2", "longitude": "-3.7058415", "no_available": 0, "address": "Calle Miguel Moya n\u00ba 1", "latitude": "40.4205886", "dock_bikes": 3, "id": 3}, {"activate": 1, "name": "Plaza Conde Suchil", "reservations_count": 0, "light": 1, "total_bases": 18, "free_bases": 5, "number": "3", "longitude": "-3.7069171", "no_available": 0, "address": "Plaza del Conde Suchil n\u00ba 2-4", "latitude": "40.4302937", "dock_bikes": 12, "id": 4}, {"activate": 1, "name": "Malasa\u00f1a", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 10, "number": "4", "longitude": "-3.7025875", "no_available": 0, "address": "Calle Manuela Malasa\u00f1a n\u00ba 5", "latitude": "40.4285524", "dock_bikes": 14, "id": 5}, {"activate": 1, "name": "Fuencarral", "reservations_count": 0, "light": 2, "total_bases": 27, "free_bases": 15, "number": "5", "longitude": "-3.70205", "no_available": 0, "address": "Calle Fuencarral n\u00ba 108", "latitude": "40.42852", "dock_bikes": 12, "id": 6}, {"activate": 1, "name": "Colegio Arquitectos", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 20, "number": "6", "longitude": "-3.6984470", "no_available": 0, "address": "Calle Hortaleza n\u00ba 63", "latitude": "40.4241480", "dock_bikes": 4, "id": 7}, {"activate": 1, "name": "Hortaleza", "reservations_count": 0, "light": 0, "total_bases": 21, "free_bases": 19, "number": "7", "longitude": "-3.6977715", "no_available": 0, "address": "Calle Hortaleza n\u00ba 75", "latitude": "40.4251906", "dock_bikes": 0, "id": 8}, {"activate": 1, "name": "Alonso Mart\u00ednez", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 17, "number": "8", "longitude": "-3.6954403", "no_available": 0, "address": "Plaza de Alonso Mart\u00ednez n\u00ba 5", "latitude": "40.4278682", "dock_bikes": 3, "id": 9}, {"activate": 1, "name": "Plaza de San Miguel", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 17, "number": "9", "longitude": "-3.7095084", "no_available": 0, "address": "Plaza de San Miguel n\u00ba 9", "latitude": "40.4156057", "dock_bikes": 5, "id": 10}, {"activate": 1, "name": "Marqu\u00e9s de la Ensenada", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 24, "number": "10", "longitude": "-3.69214", "no_available": 0, "address": "Calle Marqu\u00e9s de la Ensenada n\u00ba 16", "latitude": "40.42533", "dock_bikes": 0, "id": 11}, {"activate": 1, "name": "San Andr\u00e9s", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 14, "number": "11", "longitude": "-3.7035918", "no_available": 0, "address": "Calle San Andr\u00e9s n\u00ba 18", "latitude": "40.4269483", "dock_bikes": 10, "id": 12}, {"activate": 1, "name": "San Hermenegildo", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 7, "number": "12", "longitude": "-3.7061931", "no_available": 0, "address": "Calle San Bernardo n\u00ba 85", "latitude": "40.4284246", "dock_bikes": 16, "id": 13}, {"activate": 1, "name": "Conde Duque", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 12, "number": "13", "longitude": "-3.7104417", "no_available": 0, "address": "Calle Conde Duque n\u00ba 22", "latitude": "40.4273264", "dock_bikes": 11, "id": 14}, {"activate": 1, "name": "Ventura Rodr\u00edguez", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 19, "number": "14", "longitude": "-3.7134790", "no_available": 0, "address": "Calle Ventura Rodr\u00edguez n\u00ba 2", "latitude": "40.4260957", "dock_bikes": 4, "id": 15}, {"activate": 1, "name": "San Vicente Ferrer", "reservations_count": 0, "light": 2, "total_bases": 21, "free_bases": 8, "number": "15", "longitude": "-3.7074453", "no_available": 0, "address": "Calle San Vicente Ferrer n\u00ba 64", "latitude": "40.4262383", "dock_bikes": 12, "id": 16}, {"activate": 1, "name": "San Bernardo", "reservations_count": 0, "light": 2, "total_bases": 21, "free_bases": 14, "number": "16", "longitude": "-3.7075065", "no_available": 0, "address": "Calle San Bernardo n\u00ba 22", "latitude": "40.4230721", "dock_bikes": 7, "id": 17}, {"activate": 1, "name": "Carlos Cambronero", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 19, "number": "17", "longitude": "-3.7038312", "no_available": 0, "address": "Plaza de Carlos Cambronero n\u00ba 2", "latitude": "40.4232649", "dock_bikes": 5, "id": 18}, {"activate": 1, "name": "Plaza de Pedro Zerolo", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 21, "number": "18", "longitude": "-3.6996502", "no_available": 0, "address": "Plaza de Pedro Zerolo", "latitude": "40.4207773", "dock_bikes": 3, "id": 19}, {"activate": 1, "name": "Prim", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 16, "number": "19", "longitude": "-3.6954983", "no_available": 0, "address": "Calle Prim n\u00ba 2", "latitude": "40.4218616", "dock_bikes": 7, "id": 20}, {"activate": 1, "name": "Banco de Espa\u00f1a A", "reservations_count": 0, "light": 2, "total_bases": 30, "free_bases": 9, "number": "20a", "longitude": "-3.6954615", "no_available": 0, "address": "Calle Alcal\u00e1 n\u00ba 49", "latitude": "40.4192342", "dock_bikes": 19, "id": 21}, {"activate": 1, "name": "Jacometrezo", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 22, "number": "22", "longitude": "-3.7065376", "no_available": 0, "address": "Calle Jacometrezo n\u00ba 3", "latitude": "40.4200783", "dock_bikes": 0, "id": 25}, {"activate": 1, "name": "Santo Domingo", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 14, "number": "23", "longitude": "-3.7081246", "no_available": 0, "address": "Plaza de Santo Domingo n\u00ba 1", "latitude": "40.4202834", "dock_bikes": 8, "id": 26}, {"activate": 1, "name": "Palacio de Oriente", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 21, "number": "24", "longitude": "-3.710692", "no_available": 0, "address": "Calle Carlos III n\u00ba 1", "latitude": "40.417908", "dock_bikes": 2, "id": 27}, {"activate": 1, "name": "Plaza de Celenque A", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 21, "number": "25a", "longitude": "-3.7057791", "no_available": 0, "address": "Plaza de Celenque n\u00ba 1", "latitude": "40.4173703", "dock_bikes": 1, "id": 28}, {"activate": 1, "name": "Plaza de Celenque B", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 20, "number": "25b", "longitude": "-3.7063837", "no_available": 0, "address": "Plaza de Celenque n\u00ba 1", "latitude": "40.4172781", "dock_bikes": 3, "id": 29}, {"activate": 1, "name": "Plaza de las Salesas", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 21, "number": "26", "longitude": "-3.694475", "no_available": 0, "address": "Plaza de las Salesas 7", "latitude": "40.423855", "dock_bikes": 2, "id": 30}, {"activate": 1, "name": "Huertas", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 12, "number": "27", "longitude": "-3.6956178", "no_available": 0, "address": "Calle Jes\u00fas n\u00ba 1", "latitude": "40.4132798", "dock_bikes": 11, "id": 31}, {"activate": 1, "name": "Sevilla", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 21, "number": "28", "longitude": "-3.6984368", "no_available": 0, "address": "Calle Alcal\u00e1 n\u00ba 27", "latitude": "40.4181518", "dock_bikes": 1, "id": 32}, {"activate": 1, "name": "Marqu\u00e9s de Cubas", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 10, "number": "29", "longitude": "-3.6957355", "no_available": 0, "address": "Calle del Marqu\u00e9s de Cubas n\u00ba 25", "latitude": "40.4162619", "dock_bikes": 14, "id": 33}, {"activate": 1, "name": "San Quint\u00edn", "reservations_count": 0, "light": 0, "total_bases": 27, "free_bases": 16, "number": "30", "longitude": "-3.7115040", "no_available": 0, "address": "Calle Pav\u00eda n\u00ba 6", "latitude": "40.4192095", "dock_bikes": 2, "id": 34}, {"activate": 1, "name": "Calle Mayor", "reservations_count": 0, "light": 0, "total_bases": 27, "free_bases": 21, "number": "31", "longitude": "-3.7068969", "no_available": 0, "address": "Calle Mayor n\u00ba 20", "latitude": "40.4163638", "dock_bikes": 2, "id": 35}, {"activate": 1, "name": "Plaza de la Provincia", "reservations_count": 0, "light": 2, "total_bases": 18, "free_bases": 8, "number": "32", "longitude": "-3.7061032", "no_available": 0, "address": "Plaza de la Provincia n\u00ba 1", "latitude": "40.4150099", "dock_bikes": 8, "id": 36}, {"activate": 1, "name": "Carretas", "reservations_count": 0, "light": 0, "total_bases": 21, "free_bases": 18, "number": "33", "longitude": "-3.7030833", "no_available": 0, "address": "Calle Carretas n\u00ba 8", "latitude": "40.4165039", "dock_bikes": 3, "id": 37}, {"activate": 1, "name": "Jacinto Benavente", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 12, "number": "34", "longitude": "-3.7036825", "no_available": 0, "address": "Plaza de Jacinto Benavente", "latitude": "40.4146755", "dock_bikes": 10, "id": 38}, {"activate": 1, "name": "Plaza del Cord\u00f3n", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 6, "number": "35", "longitude": "-3.7103285", "no_available": 0, "address": "Plaza del Cord\u00f3n n\u00ba 1", "latitude": "40.4141931", "dock_bikes": 17, "id": 39}, {"activate": 1, "name": "Plaza de Ramales", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 18, "number": "36", "longitude": "-3.71183", "no_available": 0, "address": "Plaza de Ramales n\u00ba 1", "latitude": "40.4168", "dock_bikes": 5, "id": 40}, {"activate": 1, "name": "Plaza San Francisco", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 0, "number": "37", "longitude": "-3.7144964", "no_available": 0, "address": "Plaza de San Francisco n\u00ba 5", "latitude": "40.4108442", "dock_bikes": 23, "id": 41}, {"activate": 1, "name": "Plaza de los Carros", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 0, "number": "38", "longitude": "-3.7114762", "no_available": 0, "address": "Carrera de San Francisco n\u00ba 1", "latitude": "40.4114041", "dock_bikes": 24, "id": 42}, {"activate": 1, "name": "Plaza de la Cebada", "reservations_count": 0, "light": 1, "total_bases": 27, "free_bases": 0, "number": "39", "longitude": "-3.7088337", "no_available": 0, "address": "Plaza de la Cebada n\u00ba 16 ", "latitude": "40.4112744", "dock_bikes": 26, "id": 43}, {"activate": 1, "name": "Conde de Romanones", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 8, "number": "40", "longitude": "-3.7049407", "no_available": 0, "address": "Plaza del Conde de Romanones n\u00ba 9", "latitude": "40.4138846", "dock_bikes": 15, "id": 44}, {"activate": 1, "name": "Ant\u00f3n Mart\u00edn", "reservations_count": 0, "light": 3, "total_bases": 18, "free_bases": 17, "number": "41", "longitude": "-3.6991147", "no_available": 1, "address": "Calle Atocha n\u00ba 54", "latitude": "40.4122047", "dock_bikes": 0, "id": 45}, {"activate": 1, "name": "Santa Isabel", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 1, "number": "42", "longitude": "-3.6982318", "no_available": 0, "address": "Calle Santa Isabel n\u00ba 32", "latitude": "40.4107085", "dock_bikes": 20, "id": 46}, {"activate": 1, "name": "Jes\u00fas y Mar\u00eda", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 0, "number": "43", "longitude": "-3.7025024", "no_available": 0, "address": "Calle Lavapi\u00e9s n\u00ba 34-36", "latitude": "40.4101564", "dock_bikes": 22, "id": 47}, {"activate": 1, "name": "Plaza de Nelson Mandela", "reservations_count": 0, "light": 1, "total_bases": 21, "free_bases": 1, "number": "44", "longitude": "-3.7040666", "no_available": 0, "address": "Calle Mes\u00f3n de Paredes n\u00ba 35", "latitude": "40.4097617", "dock_bikes": 20, "id": 48}, {"activate": 1, "name": "Puerta de Toledo", "reservations_count": 0, "light": 2, "total_bases": 21, "free_bases": 7, "number": "45", "longitude": "-3.7110513", "no_available": 0, "address": "Glorieta de la Puerta de Toledo n\u00ba 1", "latitude": "40.4070358", "dock_bikes": 14, "id": 49}, {"activate": 1, "name": "Ribera de Curtidores", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 5, "number": "46", "longitude": "-3.7071259", "no_available": 0, "address": "Calle Ribera de Curtidores n\u00ba 28", "latitude": "40.4053153", "dock_bikes": 17, "id": 50}, {"activate": 1, "name": "Embajadores 1", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 11, "number": "47", "longitude": "-3.7028265", "no_available": 0, "address": "Glorieta de Embajadores n\u00ba 6", "latitude": "40.4047851", "dock_bikes": 11, "id": 51}, {"activate": 1, "name": "Embajadores 2", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 18, "number": "48", "longitude": "-3.7022591", "no_available": 0, "address": "Glorieta de Embajadores n\u00ba 2", "latitude": "40.4056107", "dock_bikes": 4, "id": 52}, {"activate": 1, "name": "Casa Encendida", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 15, "number": "49", "longitude": "-3.6992759", "no_available": 0, "address": "Ronda de Atocha n\u00ba 34", "latitude": "40.4060941", "dock_bikes": 9, "id": 53}, {"activate": 1, "name": "Museo Reina Sof\u00eda", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 20, "number": "50", "longitude": "-3.6933463", "no_available": 0, "address": "Calle Santa Isabel n\u00ba 57", "latitude": "40.4083684", "dock_bikes": 4, "id": 54}, {"activate": 1, "name": "Ronda de Atocha", "reservations_count": 0, "light": 0, "total_bases": 27, "free_bases": 18, "number": "51", "longitude": "-3.6935205", "no_available": 0, "address": "Ronda de Atocha n\u00ba 2", "latitude": "40.4075606", "dock_bikes": 6, "id": 55}, {"activate": 1, "name": "Plaza de Santa Ana", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 4, "number": "52", "longitude": "-3.7007164", "no_available": 0, "address": "Plaza de Santa Ana n\u00ba 10", "latitude": "40.4144226", "dock_bikes": 8, "id": 56}, {"activate": 1, "name": "Plaza de Lavapi\u00e9s", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 0, "number": "53", "longitude": "-3.7007111", "no_available": 0, "address": "Calle Valencia n\u00ba 1", "latitude": "40.4083061", "dock_bikes": 20, "id": 57}, {"activate": 1, "name": "Barcel\u00f3", "reservations_count": 0, "light": 0, "total_bases": 21, "free_bases": 16, "number": "54", "longitude": "-3.7004230", "no_available": 0, "address": "Calle Barcel\u00f3 n\u00ba 7", "latitude": "40.4266828", "dock_bikes": 5, "id": 58}, {"activate": 1, "name": "Plaza de San Ildefonso", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 7, "number": "55", "longitude": "-3.7020842", "no_available": 0, "address": "Plaza de San Ildefonso n\u00ba 3", "latitude": "40.4239757", "dock_bikes": 17, "id": 59}, {"activate": 1, "name": "Plaza del Carmen ", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 22, "number": "56", "longitude": "-3.7032414", "no_available": 0, "address": "Plaza del Carmen n\u00ba 1", "latitude": "40.4184192", "dock_bikes": 1, "id": 60}, {"activate": 1, "name": "Santa Cruz del Marcenado", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 18, "number": "57", "longitude": "-3.7126299", "no_available": 0, "address": "Calle Santa Cruz del Marcenado n\u00ba 24", "latitude": "40.4295658", "dock_bikes": 3, "id": 61}, {"activate": 1, "name": "Augusto Figueroa", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 18, "number": "58", "longitude": "-3.6978950", "no_available": 0, "address": "Calle Augusto Figueroa n\u00ba 33", "latitude": "40.4222862", "dock_bikes": 4, "id": 62}, {"activate": 1, "name": "Plaza de Juan Pujol", "reservations_count": 0, "light": 0, "total_bases": 21, "free_bases": 19, "number": "59", "longitude": "-3.7043418", "no_available": 0, "address": "Calle Esp\u00edritu Santo n\u00ba 30", "latitude": "40.4255495", "dock_bikes": 0, "id": 63}, {"activate": 1, "name": "Plaza de la Independencia", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 18, "number": "60", "longitude": "-3.6883980", "no_available": 0, "address": "Plaza de la Independencia n\u00ba 6", "latitude": "40.4197520", "dock_bikes": 4, "id": 64}, {"activate": 1, "name": "Narv\u00e1ez", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 12, "number": "61", "longitude": "-3.6752045", "no_available": 0, "address": "Calle O'Donnell n\u00ba 28", "latitude": "40.4213983", "dock_bikes": 11, "id": 65}, {"activate": 1, "name": "O'Donnell", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 15, "number": "62", "longitude": "-3.6724968", "no_available": 0, "address": "Calle O'Donnell n\u00ba 50", "latitude": "40.4213148", "dock_bikes": 6, "id": 66}, {"activate": 1, "name": "Ibiza", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 16, "number": "63", "longitude": "-3.6708959", "no_available": 0, "address": "Calle Ibiza n\u00ba 62", "latitude": "40.4179237", "dock_bikes": 5, "id": 67}, {"activate": 1, "name": "Antonio Maura", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 20, "number": "65", "longitude": "-3.6894193", "no_available": 0, "address": "Calle Antonio Maura n\u00ba 15", "latitude": "40.4166834", "dock_bikes": 4, "id": 69}, {"activate": 1, "name": "Almad\u00e9n", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 11, "number": "67", "longitude": "-3.6932250", "no_available": 0, "address": "Calle Almad\u00e9n n\u00ba 28", "latitude": "40.4108472", "dock_bikes": 12, "id": 71}, {"activate": 1, "name": "Espalter", "reservations_count": 0, "light": 2, "total_bases": 21, "free_bases": 12, "number": "68", "longitude": "-3.6912023", "no_available": 0, "address": "Calle Espalter n\u00ba 1", "latitude": "40.4128372", "dock_bikes": 9, "id": 72}, {"activate": 1, "name": "Puerta del \u00c1ngel Ca\u00eddo", "reservations_count": 0, "light": 0, "total_bases": 27, "free_bases": 21, "number": "69", "longitude": "-3.6888220", "no_available": 0, "address": "Avenida de Alfonso XII n\u00ba 54", "latitude": "40.4098080", "dock_bikes": 6, "id": 73}, {"activate": 1, "name": "Puerta del Doce de Octubre", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 8, "number": "70", "longitude": "-3.6779232", "no_available": 0, "address": "Avenida de Men\u00e9ndez Pelayo n\u00ba 63", "latitude": "40.4153053", "dock_bikes": 16, "id": 74}, {"activate": 1, "name": "Doce de Octubre", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 0, "number": "71", "longitude": "-3.6738865", "no_available": 0, "address": "Calle Doce de Octubre n\u00ba 28", "latitude": "40.4159569", "dock_bikes": 22, "id": 75}, {"activate": 1, "name": "Sainz de Baranda", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 12, "number": "72", "longitude": "-3.6691838", "no_available": 0, "address": "Calle Doctor Esquerdo n\u00ba 99", "latitude": "40.4157413", "dock_bikes": 11, "id": 76}, {"activate": 1, "name": "Plaza de los Astros", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 15, "number": "73", "longitude": "-3.6689089", "no_available": 0, "address": "Avenida de Nazaret n\u00ba 7", "latitude": "40.4114475", "dock_bikes": 8, "id": 77}, {"activate": 1, "name": "Puerta del Pac\u00edfico ", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 6, "number": "74", "longitude": "-3.6766813", "no_available": 0, "address": "Avenida de Men\u00e9ndez Pelayo n\u00ba 73", "latitude": "40.4117627", "dock_bikes": 16, "id": 78}, {"activate": 1, "name": "Men\u00e9ndez Pelayo", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 1, "number": "75", "longitude": "-3.6784838", "no_available": 0, "address": "Avenida de Men\u00e9ndez Pelayo n\u00ba 90", "latitude": "40.4082805", "dock_bikes": 23, "id": 79}, {"activate": 1, "name": "Puerta de Mariano de Cavia", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 3, "number": "76", "longitude": "-3.6751352", "no_available": 0, "address": "Avenida del Mediterr\u00e1neo n\u00ba 19", "latitude": "40.4071304", "dock_bikes": 20, "id": 80}, {"activate": 1, "name": "Conde de Casal", "reservations_count": 0, "light": 2, "total_bases": 22, "free_bases": 7, "number": "77", "longitude": "-3.6704220", "no_available": 0, "address": "Plaza del Conde de Casal n\u00ba 8", "latitude": "40.4063500", "dock_bikes": 15, "id": 81}, {"activate": 1, "name": "Pedro Bosch", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 5, "number": "78", "longitude": "-3.674400", "no_available": 0, "address": "Calle Cerro de la Plata n\u00ba 2", "latitude": "40.400789", "dock_bikes": 19, "id": 82}, {"activate": 1, "name": "Puerta de Granada", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 0, "number": "79", "longitude": "-3.6803874", "no_available": 0, "address": "Avenida de Men\u00e9ndez Pelayo n\u00ba 38", "latitude": "40.4051451", "dock_bikes": 23, "id": 83}, {"activate": 1, "name": "Atocha A", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 18, "number": "80a", "longitude": "-3.6902255", "no_available": 0, "address": "Paseo de la Infanta Isabel n\u00ba 3", "latitude": "40.4075685", "dock_bikes": 4, "id": 84}, {"activate": 1, "name": "Atocha B", "reservations_count": 0, "light": 0, "total_bases": 27, "free_bases": 24, "number": "80b", "longitude": "-3.6901234", "no_available": 0, "address": "Paseo de la Infanta Isabel n\u00ba 3", "latitude": "40.4074902", "dock_bikes": 2, "id": 85}, {"activate": 1, "name": "Cuesta de Moyano", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 18, "number": "81", "longitude": "-3.6919870", "no_available": 0, "address": "Plaza del Emperador Carlos V n\u00ba 11", "latitude": "40.4092970", "dock_bikes": 2, "id": 86}, {"activate": 1, "name": "Ni\u00f1o Jes\u00fas", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 15, "number": "82", "longitude": "-3.6697526", "no_available": 0, "address": "Calle Doctor Esquerdo n\u00ba 161", "latitude": "40.4084556", "dock_bikes": 9, "id": 87}, {"activate": 1, "name": "P\u00edo Baroja", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 4, "number": "83", "longitude": "-3.6745323", "no_available": 0, "address": "Calle P\u00edo Baroja n\u00ba 10", "latitude": "40.4132995", "dock_bikes": 18, "id": 88}, {"activate": 1, "name": "Valderribas", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 1, "number": "84", "longitude": "-3.6726019", "no_available": 0, "address": "Calle Doctor Esquerdo n\u00ba 191", "latitude": "40.4032501", "dock_bikes": 22, "id": 89}, {"activate": 1, "name": "Puerta de Madrid", "reservations_count": 0, "light": 2, "total_bases": 27, "free_bases": 17, "number": "85", "longitude": "-3.6800080", "no_available": 0, "address": "Avenida de Men\u00e9ndez Pelayo n\u00ba 11", "latitude": "40.4215010", "dock_bikes": 8, "id": 90}, {"activate": 1, "name": "Cibeles", "reservations_count": 0, "light": 0, "total_bases": 21, "free_bases": 14, "number": "86", "longitude": "-3.6933498", "no_available": 0, "address": "Paseo del Prado n\u00ba 1", "latitude": "40.4186516", "dock_bikes": 5, "id": 91}, {"activate": 1, "name": "Ayala", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 23, "number": "87", "longitude": "-3.6832566", "no_available": 0, "address": "Calle Ayala n\u00ba 44", "latitude": "40.4277360", "dock_bikes": 1, "id": 92}, {"activate": 1, "name": "Embajada de Italia", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 22, "number": "88", "longitude": "-3.6838303", "no_available": 0, "address": "Calle Vel\u00e1zquez n\u00ba 75", "latitude": "40.4313576", "dock_bikes": 2, "id": 93}, {"activate": 1, "name": "Conde Pe\u00f1alver", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 12, "number": "89", "longitude": "-3.6752024", "no_available": 0, "address": "Calle Ayala n\u00ba 102", "latitude": "40.4272582", "dock_bikes": 11, "id": 94}, {"activate": 1, "name": "General Pardi\u00f1as", "reservations_count": 0, "light": 0, "total_bases": 30, "free_bases": 28, "number": "90", "longitude": "-3.6837876", "no_available": 0, "address": "Calle Goya n\u00ba 18", "latitude": "40.4250361", "dock_bikes": 2, "id": 95}, {"activate": 1, "name": "Pr\u00edncipe de Vergara", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 9, "number": "91", "longitude": "-3.6787441", "no_available": 0, "address": "Calle Hermosilla n\u00ba 70", "latitude": "40.4261340", "dock_bikes": 10, "id": 96}, {"activate": 1, "name": "Claudio Coello", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 19, "number": "92", "longitude": "-3.6865463", "no_available": 0, "address": "Calle Claudio Coello n\u00ba 45", "latitude": "40.4262945", "dock_bikes": 5, "id": 97}, {"activate": 1, "name": "Plaza de Col\u00f3n", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 24, "number": "93", "longitude": "-3.6893698", "no_available": 0, "address": "Calle Goya n\u00ba 1", "latitude": "40.4257046", "dock_bikes": 0, "id": 98}, {"activate": 1, "name": "Biblioteca Nacional", "reservations_count": 0, "light": 0, "total_bases": 21, "free_bases": 15, "number": "94", "longitude": "-3.690756", "no_available": 0, "address": "Paseo de Recoletos n\u00ba 32-34", "latitude": "40.4232153", "dock_bikes": 1, "id": 99}, {"activate": 1, "name": "Villanueva", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 21, "number": "95", "longitude": "-3.6870548", "no_available": 0, "address": "Calle Claudio Coello n\u00ba 19", "latitude": "40.4226584", "dock_bikes": 2, "id": 100}, {"activate": 1, "name": "Castell\u00f3", "reservations_count": 0, "light": 2, "total_bases": 27, "free_bases": 14, "number": "96", "longitude": "-3.6821793", "no_available": 0, "address": "Calle Alcal\u00e1 n\u00ba 111", "latitude": "40.4220640", "dock_bikes": 10, "id": 101}, {"activate": 1, "name": "Alcal\u00e1", "reservations_count": 0, "light": 2, "total_bases": 27, "free_bases": 15, "number": "97", "longitude": "-3.6805189", "no_available": 0, "address": "Avenida de Men\u00e9ndez Pelayo n\u00ba 3", "latitude": "40.4222969", "dock_bikes": 12, "id": 102}, {"activate": 1, "name": "Plaza de Felipe II", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 6, "number": "98", "longitude": "-3.6758383", "no_available": 0, "address": "Plaza de Felipe II", "latitude": "40.4239447", "dock_bikes": 16, "id": 103}, {"activate": 1, "name": "Alc\u00e1ntara", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 12, "number": "99", "longitude": "-3.6738714", "no_available": 0, "address": "Calle Alc\u00e1ntara n\u00ba 2", "latitude": "40.4261851", "dock_bikes": 10, "id": 104}, {"activate": 1, "name": "Palacio de Deportes", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 11, "number": "100", "longitude": "-3.67384", "no_available": 0, "address": "Calle Goya n\u00ba 99", "latitude": "40.42478", "dock_bikes": 9, "id": 105}, {"activate": 1, "name": "Jorge Juan", "reservations_count": 0, "light": 3, "total_bases": 24, "free_bases": 0, "number": "101", "longitude": "-3.6691524", "no_available": 1, "address": "Calle Jorge Juan n\u00ba 131", "latitude": "40.4231526", "dock_bikes": 0, "id": 106}, {"activate": 1, "name": "Vel\u00e1zquez", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 16, "number": "102", "longitude": "-3.6840229", "no_available": 0, "address": "Calle Alcal\u00e1 n\u00ba 95", "latitude": "40.4211802", "dock_bikes": 4, "id": 107}, {"activate": 1, "name": "Ortega y Gasset", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 20, "number": "103", "longitude": "-3.68653", "no_available": 0, "address": "Calle Ortega y Gasset n\u00ba 6", "latitude": "40.43037", "dock_bikes": 4, "id": 108}, {"activate": 1, "name": "Castellana", "reservations_count": 0, "light": 0, "total_bases": 21, "free_bases": 17, "number": "104", "longitude": "-3.68958", "no_available": 0, "address": "Paseo de la Castellana n\u00ba 4", "latitude": "40.42677", "dock_bikes": 4, "id": 109}, {"activate": 1, "name": "Serrano", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 24, "number": "105", "longitude": "-3.68745", "no_available": 0, "address": "Calle Serrano n\u00ba 54", "latitude": "40.42699", "dock_bikes": 0, "id": 110}, {"activate": 1, "name": "Col\u00f3n A", "reservations_count": 0, "light": 0, "total_bases": 18, "free_bases": 15, "number": "106a", "longitude": "-3.6877227", "no_available": 0, "address": "Calle Serrano n\u00ba 34", "latitude": "40.4251002", "dock_bikes": 1, "id": 111}, {"activate": 1, "name": "Col\u00f3n B", "reservations_count": 0, "light": 0, "total_bases": 18, "free_bases": 18, "number": "106b", "longitude": "-3.6877450", "no_available": 0, "address": "Calle Serrano n\u00ba 34", "latitude": "40.4249630", "dock_bikes": 0, "id": 112}, {"activate": 1, "name": "Columela", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 22, "number": "107", "longitude": "-3.6884369", "no_available": 0, "address": "Calle Serrano n\u00ba 6", "latitude": "40.4215246", "dock_bikes": 2, "id": 113}, {"activate": 1, "name": "M\u00e1rtires Concepcionistas", "reservations_count": 0, "light": 0, "total_bases": 27, "free_bases": 22, "number": "108", "longitude": "-3.6706024", "no_available": 0, "address": "Calle M\u00e1rtires Concepcionistas n\u00ba 2", "latitude": "40.4273005", "dock_bikes": 3, "id": 114}, {"activate": 1, "name": "Marqu\u00e9s de Salamanca", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 20, "number": "109", "longitude": "-3.6816402", "no_available": 0, "address": "Calle Jos\u00e9 Ortega y Gasset n\u00ba 30", "latitude": "40.4300481", "dock_bikes": 4, "id": 115}, {"activate": 1, "name": "Moncloa", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 20, "number": "110", "longitude": "-3.7200845", "no_available": 0, "address": "Paseo de Moret n\u00ba 2", "latitude": "40.4347895", "dock_bikes": 3, "id": 116}, {"activate": 1, "name": "Arcipreste de Hita A", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 19, "number": "111a", "longitude": "-3.7175435", "no_available": 0, "address": "Calle Arcipreste de Hita n\u00ba 12", "latitude": "40.4337322", "dock_bikes": 3, "id": 117}, {"activate": 1, "name": "Arcipreste de Hita B", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 20, "number": "111b", "longitude": "-3.7179687", "no_available": 0, "address": "Calle Arcipreste de Hita n\u00ba 12", "latitude": "40.4341006", "dock_bikes": 2, "id": 118}, {"activate": 1, "name": "Paseo de Moret", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 12, "number": "112", "longitude": "-3.7246532", "no_available": 0, "address": "Paseo de Moret n\u00ba 11", "latitude": "40.4325991", "dock_bikes": 12, "id": 119}, {"activate": 1, "name": "Pintor Rosales", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 21, "number": "113", "longitude": "-3.7205129", "no_available": 0, "address": "Paseo del Pintor Rosales n\u00ba 38", "latitude": "40.4276570", "dock_bikes": 3, "id": 120}, {"activate": 1, "name": "Quintana", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 21, "number": "114", "longitude": "-3.7174158", "no_available": 0, "address": "Calle Quintana n\u00ba 11-13", "latitude": "40.4277456", "dock_bikes": 3, "id": 121}, {"activate": 1, "name": "Ferraz", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 19, "number": "115", "longitude": "-3.7170448", "no_available": 0, "address": "Calle Evaristo San Miguel n\u00ba 2", "latitude": "40.4253944", "dock_bikes": 5, "id": 122}, {"activate": 1, "name": "Plaza de Espa\u00f1a A", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 23, "number": "116a", "longitude": "-3.7116030", "no_available": 0, "address": "Plaza de Espa\u00f1a", "latitude": "40.4240200", "dock_bikes": 0, "id": 123}, {"activate": 1, "name": "Plaza de Espa\u00f1a B", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 23, "number": "116b", "longitude": "-3.7117030", "no_available": 0, "address": "Plaza de Espa\u00f1a", "latitude": "40.4241200", "dock_bikes": 0, "id": 124}, {"activate": 1, "name": "Altamirano", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 17, "number": "117", "longitude": "-3.7188898", "no_available": 0, "address": "Calle Altamirano n\u00ba 24", "latitude": "40.4309797", "dock_bikes": 5, "id": 125}, {"activate": 1, "name": "Juan Mart\u00edn", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 1, "number": "118", "longitude": "-3.6882407", "no_available": 0, "address": "Calle Juan Mart\u00edn el Empecinado n\u00ba 14", "latitude": "40.4007810", "dock_bikes": 22, "id": 126}, {"activate": 1, "name": "M\u00e9ndez \u00c1lvaro", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 11, "number": "119", "longitude": "-3.6863218", "no_available": 0, "address": "Calle M\u00e9ndez \u00c1lvaro n\u00ba 11", "latitude": "40.4013216", "dock_bikes": 12, "id": 127}, {"activate": 1, "name": "Palos de la Frontera", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 14, "number": "120", "longitude": "-3.6944768", "no_available": 0, "address": "Calle Palos de la Frontera n\u00ba 40", "latitude": "40.4032208", "dock_bikes": 9, "id": 128}, {"activate": 1, "name": "Santa Mar\u00eda de la Cabeza", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 7, "number": "121", "longitude": "-3.6987665", "no_available": 0, "address": "Paseo de Santa Mar\u00eda de la Cabeza n\u00ba 58", "latitude": "40.4017926", "dock_bikes": 13, "id": 129}, {"activate": 1, "name": "Santa Engracia 14", "reservations_count": 1, "light": 0, "total_bases": 24, "free_bases": 18, "number": "122", "longitude": "-3.6967169", "no_available": 0, "address": "Calle Santa Engracia n\u00ba 14", "latitude": "40.4291992", "dock_bikes": 3, "id": 130}, {"activate": 1, "name": "Guzm\u00e1n el Bueno", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 19, "number": "123", "longitude": "-3.7133412", "no_available": 0, "address": "Calle Guzm\u00e1n el Bueno n\u00ba 2", "latitude": "40.4306458", "dock_bikes": 4, "id": 131}, {"activate": 1, "name": "Manuel Silvela", "reservations_count": 1, "light": 2, "total_bases": 24, "free_bases": 14, "number": "124", "longitude": "-3.6993465", "no_available": 0, "address": "Calle Manuel Silvela n\u00ba 20", "latitude": "40.4309524", "dock_bikes": 7, "id": 169}, {"activate": 1, "name": "General Mart\u00ednez Campos ", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 17, "number": "125", "longitude": "-3.6948626", "no_available": 0, "address": "Calle Fern\u00e1ndez de la Hoz n\u00ba 29", "latitude": "40.4352850", "dock_bikes": 5, "id": 164}, {"activate": 1, "name": "General \u00c1lvarez de Castro", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 13, "number": "126", "longitude": "-3.7015686", "no_available": 0, "address": "Calle del General \u00c1lvarez de Castro n\u00ba 2", "latitude": "40.4344731", "dock_bikes": 10, "id": 163}, {"activate": 1, "name": "Fernando el Cat\u00f3lico", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 0, "number": "127", "longitude": "-3.7084390", "no_available": 0, "address": "Calle Fernando el Cat\u00f3lico n\u00ba 19", "latitude": "40.4338516", "dock_bikes": 23, "id": 168}, {"activate": 1, "name": "Cea Berm\u00fadez", "reservations_count": 0, "light": 0, "total_bases": 27, "free_bases": 18, "number": "128", "longitude": "-3.7154329", "no_available": 0, "address": "Calle Cea Berm\u00fadez n\u00ba 59", "latitude": "40.4389940", "dock_bikes": 5, "id": 160}, {"activate": 1, "name": "Jos\u00e9 Abascal", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 9, "number": "129", "longitude": "-3.6982353", "no_available": 0, "address": "Calle Jos\u00e9 Abascal n\u00ba 33", "latitude": "40.4383865", "dock_bikes": 12, "id": 161}, {"activate": 1, "name": "Santa Engracia 127", "reservations_count": 0, "light": 2, "total_bases": 20, "free_bases": 12, "number": "130", "longitude": "-3.70164", "no_available": 0, "address": "Calle Santa Engracia n\u00ba 127", "latitude": "40.44156", "dock_bikes": 6, "id": 157}, {"activate": 1, "name": "Bravo Murillo 44", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 17, "number": "131", "longitude": "-3.7040344", "no_available": 0, "address": "Calle Bravo Murillo n\u00ba 44", "latitude": "40.4418402", "dock_bikes": 7, "id": 156}, {"activate": 1, "name": "Glorieta de los Cuatro Caminos", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 2, "number": "132", "longitude": "-3.7036675", "no_available": 0, "address": "Calle Santa Engracia n\u00ba 168", "latitude": "40.4463667", "dock_bikes": 18, "id": 149}, {"activate": 1, "name": "Raimundo Fern\u00e1ndez", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 8, "number": "133", "longitude": "-3.7001669", "no_available": 0, "address": "Calle Raimundo Fern\u00e1ndez Villaverde n\u00ba 33-35", "latitude": "40.4471250", "dock_bikes": 15, "id": 150}, {"activate": 1, "name": "Agust\u00edn de Betancourt", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 16, "number": "134", "longitude": "-3.6956047", "no_available": 0, "address": "Calle Mar\u00eda de Guzm\u00e1n n\u00ba 58", "latitude": "40.4440297", "dock_bikes": 6, "id": 153}, {"activate": 1, "name": "Plaza de San Juan de la Cruz", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 11, "number": "135", "longitude": "-3.6927821", "no_available": 0, "address": "Plaza de San Juan de la Cruz n\u00ba 11", "latitude": "40.4415974", "dock_bikes": 12, "id": 158}, {"activate": 1, "name": "Castellana 106", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 23, "number": "136", "longitude": "-3.6908610", "no_available": 0, "address": "Paseo de la Castellana n\u00ba 106", "latitude": "40.4453307", "dock_bikes": 0, "id": 151}, {"activate": 1, "name": "Plaza de la Rep\u00fablica Argentina", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 22, "number": "137", "longitude": "-3.6853312", "no_available": 0, "address": "Plaza de la Rep\u00fablica Argentina n\u00ba 6-7", "latitude": "40.4454110", "dock_bikes": 1, "id": 152}, {"activate": 1, "name": "Paseo de la Castellana con Raimundo Fern\u00e1ndez", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 19, "number": "138", "longitude": "-3.6917932", "no_available": 0, "address": "Paseo de la Castellana n\u00ba 67", "latitude": "40.4457414", "dock_bikes": 2, "id": 154}, {"activate": 1, "name": "Jos\u00e9 Guti\u00e9rrez Abascal ", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 20, "number": "139", "longitude": "-3.6907784", "no_available": 0, "address": "Calle Jos\u00e9 Guti\u00e9rrez Abascal n\u00ba 2", "latitude": "40.4396792", "dock_bikes": 4, "id": 159}, {"activate": 1, "name": "Vel\u00e1zquez 130", "reservations_count": 0, "light": 0, "total_bases": 27, "free_bases": 26, "number": "140", "longitude": "-3.6831578", "no_available": 0, "address": "Calle Vel\u00e1zquez n\u00ba 130", "latitude": "40.4385127", "dock_bikes": 1, "id": 162}, {"activate": 1, "name": "P\u00ba Castellana - Glorieta de Emilio Castelar", "reservations_count": 0, "light": 0, "total_bases": 12, "free_bases": 8, "number": "141", "longitude": "-3.6892368", "no_available": 0, "address": "Paseo de la Castellana n\u00ba 43", "latitude": "40.4355143", "dock_bikes": 3, "id": 165}, {"activate": 1, "name": "Castellana 42", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 17, "number": "142", "longitude": "-3.6879154", "no_available": 0, "address": "Paseo de la Castellana n\u00ba 42", "latitude": "40.4334087", "dock_bikes": 4, "id": 167}, {"activate": 1, "name": "Diego de Le\u00f3n 52", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 18, "number": "143", "longitude": "-3.6784920", "no_available": 0, "address": "Calle Diego de Le\u00f3n n\u00ba 52", "latitude": "40.4345973", "dock_bikes": 6, "id": 166}, {"activate": 1, "name": "Juan Bravo 50", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 19, "number": "144", "longitude": "-3.6758555", "no_available": 0, "address": "Calle Juan Bravo n\u00ba 50", "latitude": "40.4323655", "dock_bikes": 5, "id": 170}, {"activate": 1, "name": "Ortega y Gasset 87", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 5, "number": "145", "longitude": "-3.6712823", "no_available": 0, "address": "Calle Ortega y Gasset n\u00ba 87", "latitude": "40.4298870", "dock_bikes": 19, "id": 171}, {"activate": 1, "name": "Mar\u00eda Francisca 1", "reservations_count": 0, "light": 1, "total_bases": 18, "free_bases": 5, "number": "146", "longitude": "-3.6787169", "no_available": 0, "address": "Calle Mar\u00eda Francisca n\u00ba 1", "latitude": "40.4442258", "dock_bikes": 12, "id": 155}, {"activate": 1, "name": "Doctor Arce 45", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 20, "number": "147", "longitude": "-3.6797296", "no_available": 0, "address": "Avenida del Doctor Arce n\u00ba 45", "latitude": "40.4483269", "dock_bikes": 3, "id": 148}, {"activate": 1, "name": "Serrano 210", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 13, "number": "148", "longitude": "-3.6817962", "no_available": 0, "address": "Calle Serrano n\u00ba 210", "latitude": "40.4510188", "dock_bikes": 10, "id": 144}, {"activate": 1, "name": "Paseo de la Habana 42", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 7, "number": "149", "longitude": "-3.6881689", "no_available": 0, "address": "Paseo de la Habana n\u00ba 42", "latitude": "40.4498613", "dock_bikes": 17, "id": 146}, {"activate": 1, "name": "Castellana frente a Hermanos Pinz\u00f3n", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 20, "number": "150", "longitude": "-3.6905604", "no_available": 0, "address": "Paseo de la Castellana n\u00ba 124", "latitude": "40.4488924", "dock_bikes": 2, "id": 147}, {"activate": 1, "name": "Orense 12", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 19, "number": "151", "longitude": "-3.6952943", "no_available": 0, "address": "Calle Orense n\u00ba 12", "latitude": "40.4489101", "dock_bikes": 1, "id": 145}, {"activate": 1, "name": "General Per\u00f3n 1", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 4, "number": "152", "longitude": "-3.6990077", "no_available": 0, "address": "Avenida del General Per\u00f3n n\u00ba 1-4", "latitude": "40.4527164", "dock_bikes": 19, "id": 142}, {"activate": 1, "name": "General Per\u00f3n con Poeta Joan Maragall", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 18, "number": "153", "longitude": "-3.693085", "no_available": 0, "address": "Avenida del General Per\u00f3n n\u00ba 36", "latitude": "40.4522454", "dock_bikes": 5, "id": 143}, {"activate": 1, "name": "Orense 36", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 19, "number": "154", "longitude": "-3.6946218", "no_available": 0, "address": "Calle Orense n\u00ba 36", "latitude": "40.4548456", "dock_bikes": 4, "id": 141}, {"activate": 1, "name": "San Germ\u00e1n", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 17, "number": "155", "longitude": "-3.7009675", "no_available": 0, "address": "Calle del General Yag\u00fce n\u00ba 57", "latitude": "40.4572824", "dock_bikes": 5, "id": 139}, {"activate": 1, "name": "Sor \u00c1ngela de la Cruz", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 21, "number": "156", "longitude": "-3.6915330", "no_available": 0, "address": "Calle Sor \u00c1ngela de la Cruz n\u00ba 2", "latitude": "40.4592351", "dock_bikes": 1, "id": 140}, {"activate": 1, "name": "Castellana 164", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 20, "number": "157", "longitude": "-3.6894151", "no_available": 0, "address": "Paseo de la Castellana n\u00ba 164", "latitude": "40.4591366", "dock_bikes": 4, "id": 137}, {"activate": 1, "name": "Alberto Alcocer", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 15, "number": "158", "longitude": "-3.6847150", "no_available": 0, "address": "Avenida de Alberto Alcocer n\u00ba 22", "latitude": "40.4585318", "dock_bikes": 7, "id": 138}, {"activate": 1, "name": "Paseo de la Habana 63", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 14, "number": "159", "longitude": "-3.6835926", "no_available": 0, "address": "Paseo de la Habana n\u00ba 63", "latitude": "40.4543852", "dock_bikes": 9, "id": 173}, {"activate": 1, "name": "Colombia", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 16, "number": "160", "longitude": "-3.6763439", "no_available": 0, "address": "Calle Colombia n\u00ba 7", "latitude": "40.4572466", "dock_bikes": 7, "id": 172}, {"activate": 1, "name": "Paseo de la Florida", "reservations_count": 0, "light": 1, "total_bases": 27, "free_bases": 5, "number": "161", "longitude": "-3.7224983", "no_available": 0, "address": "Paseo de la Florida n\u00ba 8", "latitude": "40.4219665", "dock_bikes": 19, "id": 132}, {"activate": 1, "name": "Metro Pir\u00e1mides", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 2, "number": "162", "longitude": "-3.7108108", "no_available": 0, "address": "Paseo de los Olmos n\u00ba 28", "latitude": "40.4034076", "dock_bikes": 21, "id": 133}, {"activate": 1, "name": "Paseo de la Esperanza", "reservations_count": 0, "light": 2, "total_bases": 21, "free_bases": 9, "number": "163", "longitude": "-3.7064516", "no_available": 0, "address": "Paseo de la Esperanza n\u00ba 2", "latitude": "40.4035988", "dock_bikes": 11, "id": 134}, {"activate": 1, "name": "Paseo de las Delicias", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 3, "number": "164", "longitude": "-3.6945025", "no_available": 0, "address": "Paseo de las Delicias n\u00ba 92-94", "latitude": "40.3972616", "dock_bikes": 20, "id": 136}, {"activate": 1, "name": "Entrada Matadero", "reservations_count": 0, "light": 2, "total_bases": 27, "free_bases": 9, "number": "165", "longitude": "-3.6975708", "no_available": 0, "address": "Paseo de la Chopera n\u00ba 14", "latitude": "40.3928821", "dock_bikes": 16, "id": 135}, {"activate": 1, "name": "Segovia 26", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 18, "number": "166", "longitude": "-3.7135833", "no_available": 0, "address": "Calle Segovia, 26", "latitude": "40.4138333", "dock_bikes": 6, "id": 174}, {"activate": 1, "name": "Segovia 45", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 3, "number": "167", "longitude": "-3.717487", "no_available": 0, "address": "Calle Segovia, 45", "latitude": "40.413736", "dock_bikes": 21, "id": 175}, {"activate": 1, "name": "Batalla del Salado", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 9, "number": "168", "longitude": "-3.6959166", "no_available": 0, "address": "P\u00ba Sta. M\u00aa Cabeza, 29-31", "latitude": "40.4044722", "dock_bikes": 15, "id": 176}, {"activate": 1, "name": "Pir\u00e1mides", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 0, "number": "169", "longitude": "-3.7138055", "no_available": 0, "address": "Toledo, 181", "latitude": "40.4016111", "dock_bikes": 23, "id": 177}, {"activate": 1, "name": "Paseo de la Esperanza", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 4, "number": "170", "longitude": "-3.7043611", "no_available": 0, "address": "P\u00ba Esperanza, 21", "latitude": "40.401", "dock_bikes": 20, "id": 178}, {"activate": 1, "name": "Embajadores-C\u00e1ceres", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 9, "number": "171", "longitude": "-3.6983888", "no_available": 0, "address": "Embajadores, 101", "latitude": "40.39975", "dock_bikes": 13, "id": 179}, {"activate": 1, "name": "Delicias", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 15, "number": "172", "longitude": "-3.6935833", "no_available": 0, "address": "Bustamante, 1", "latitude": "40.4008888", "dock_bikes": 8, "id": 180}, {"activate": 1, "name": "Puente Praga", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 0, "number": "173", "longitude": "-3.7023918", "no_available": 0, "address": "P\u00ba Esperanza, 59", "latitude": "40.3981798", "dock_bikes": 18, "id": 181}, {"activate": 1, "name": "Madrid R\u00edo-Yeser\u00edas", "reservations_count": 0, "light": 1, "total_bases": 27, "free_bases": 5, "number": "174", "longitude": "-3.7065277", "no_available": 0, "address": "P\u00ba Yeser\u00edas 45", "latitude": "40.3974444", "dock_bikes": 19, "id": 182}, {"activate": 1, "name": "Jaime el Conquistador", "reservations_count": 0, "light": 1, "total_bases": 21, "free_bases": 0, "number": "175", "longitude": "-3.6983055", "no_available": 0, "address": "Jaime I el Conquistador, 30", "latitude": "40.3962222", "dock_bikes": 18, "id": 183}, {"activate": 1, "name": "Beata", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 12, "number": "176", "longitude": "-3.6937222", "no_available": 0, "address": "Alicante, 3", "latitude": "40.39425", "dock_bikes": 11, "id": 184}, {"activate": 1, "name": "Legazpi", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 6, "number": "177", "longitude": "-3.6941944", "no_available": 0, "address": "Bolivar, 3", "latitude": "40.3914722", "dock_bikes": 16, "id": 185}, {"activate": 1, "name": "Junta Municipal Retiro", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 1, "number": "178", "longitude": "-3.6788343", "no_available": 0, "address": "Le\u00f3n Gil de Palacios, 5", "latitude": "40.4029202", "dock_bikes": 22, "id": 186}, {"activate": 1, "name": "Puente de Vallecas", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 4, "number": "179", "longitude": "-3.66925", "no_available": 0, "address": "Avda. Albufera 6 con Antonia Calas ", "latitude": "40.3979722", "dock_bikes": 19, "id": 187}, {"activate": 1, "name": "M\u00e9ndez \u00c1lvaro", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 5, "number": "180", "longitude": "-3.6758888", "no_available": 0, "address": "Retama, 5", "latitude": "40.3941388", "dock_bikes": 19, "id": 188}, {"activate": 1, "name": "Retiro-Ibiza", "reservations_count": 0, "light": 2, "total_bases": 27, "free_bases": 10, "number": "181", "longitude": "-3.6766111", "no_available": 0, "address": "Ibiza, 18", "latitude": "40.41825", "dock_bikes": 15, "id": 189}, {"activate": 1, "name": "Parque Roma", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 3, "number": "182", "longitude": "-3.6657777", "no_available": 0, "address": "Sainz de Baranda, frente al n\u00ba 107", "latitude": "40.4186666", "dock_bikes": 19, "id": 190}, {"activate": 1, "name": "Doctor Esquerdo-Hermosilla", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 14, "number": "183", "longitude": "-3.669339", "no_available": 0, "address": "Hermosilla, 135", "latitude": "40.425635", "dock_bikes": 9, "id": 191}, {"activate": 1, "name": "Marqu\u00e9s de Zafra", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 12, "number": "184", "longitude": "-3.6653055", "no_available": 0, "address": "P\u00ba Marqu\u00e9s de Zafra, 24", "latitude": "40.426", "dock_bikes": 7, "id": 192}, {"activate": 1, "name": "Quinta Fuente del Berro", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 3, "number": "185", "longitude": "-3.6615277", "no_available": 0, "address": "Sancho D\u00e1vila esq. P\u00ba Marqu\u00e9s de Zafra", "latitude": "40.4250833", "dock_bikes": 14, "id": 193}, {"activate": 1, "name": "General Pardi\u00f1as", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 18, "number": "186", "longitude": "-3.6782777", "no_available": 0, "address": "General Pardi\u00f1as, 56", "latitude": "40.4290555", "dock_bikes": 5, "id": 194}, {"activate": 1, "name": "Alcal\u00e1-Ventas", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 1, "number": "187", "longitude": "-3.6638888", "no_available": 0, "address": "Alejandro Gonz\u00e1lez, 1", "latitude": "40.4301666", "dock_bikes": 19, "id": 195}, {"activate": 1, "name": "Puente de Ventas", "reservations_count": 0, "light": 1, "total_bases": 27, "free_bases": 4, "number": "188", "longitude": "-3.6590555", "no_available": 0, "address": "Avda. Daroca, 2", "latitude": "40.4311944", "dock_bikes": 23, "id": 196}, {"activate": 1, "name": "Avenida Donostiarra", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 5, "number": "189", "longitude": "-3.6554918", "no_available": 0, "address": "Virgen de la Alegria 18", "latitude": "40.4351256", "dock_bikes": 19, "id": 197}, {"activate": 1, "name": "Vel\u00e1zquez-Diego de Le\u00f3n", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 18, "number": "190", "longitude": "-3.6835833", "no_available": 0, "address": "Vel\u00e1zquez, 91", "latitude": "40.4343333", "dock_bikes": 6, "id": 198}, {"activate": 1, "name": "Diego de le\u00f3n", "reservations_count": 0, "light": 0, "total_bases": 18, "free_bases": 13, "number": "191", "longitude": "-3.674", "no_available": 0, "address": "Alonso Heredia, 1", "latitude": "40.4346388", "dock_bikes": 3, "id": 199}, {"activate": 1, "name": "Avenida de los Toreros", "reservations_count": 0, "light": 1, "total_bases": 27, "free_bases": 5, "number": "192", "longitude": "-3.6714166", "no_available": 0, "address": "Avda. Toreros,3", "latitude": "40.4318611", "dock_bikes": 20, "id": 200}, {"activate": 1, "name": "Ventas", "reservations_count": 0, "light": 2, "total_bases": 27, "free_bases": 11, "number": "193", "longitude": "-3.6655555", "no_available": 0, "address": "Avda. Toreros, 49", "latitude": "40.4324722", "dock_bikes": 14, "id": 201}, {"activate": 1, "name": "Avenida de Am\u00e9rica 2", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 9, "number": "194", "longitude": "-3.6772222", "no_available": 0, "address": "Francisco Silvela 86", "latitude": "40.43725", "dock_bikes": 14, "id": 202}, {"activate": 1, "name": "Avenida de Am\u00e9rica 1", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 15, "number": "195", "longitude": "-3.6754444", "no_available": 0, "address": "Padre Indalecio Hernandez, 1", "latitude": "40.4392222", "dock_bikes": 7, "id": 203}, {"activate": 1, "name": "Prosperidad", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 8, "number": "196", "longitude": "-3.6751388", "no_available": 0, "address": "Suero de Qui\u00f1ones, 2", "latitude": "40.4441388", "dock_bikes": 15, "id": 204}, {"activate": 1, "name": "Parque Berl\u00edn", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 12, "number": "197", "longitude": "-3.6780555", "no_available": 0, "address": "Avenida Concha Espina, 34 ", "latitude": "40.4515833", "dock_bikes": 12, "id": 205}, {"activate": 1, "name": "Rub\u00e9n Dar\u00edo", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 17, "number": "198", "longitude": "-3.6915555", "no_available": 0, "address": "Gta. Rub\u00e9n Dar\u00edo 2", "latitude": "40.4331111", "dock_bikes": 6, "id": 206}, {"activate": 1, "name": "Fernando el Cat\u00f3lico", "reservations_count": 0, "light": 0, "total_bases": 27, "free_bases": 21, "number": "199", "longitude": "-3.7149166", "no_available": 0, "address": "Fernando el Cat\u00f3lico, 61", "latitude": "40.4343611", "dock_bikes": 4, "id": 207}, {"activate": 1, "name": "Quevedo", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 13, "number": "200", "longitude": "-3.7046666", "no_available": 0, "address": "Gta. Quevedo, 5", "latitude": "40.4336944", "dock_bikes": 8, "id": 208}, {"activate": 1, "name": "Canal", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 20, "number": "201", "longitude": "-3.7038333", "no_available": 0, "address": "Jos\u00e9 Abascal, frene al n\u00ba 2", "latitude": "40.4386944", "dock_bikes": 4, "id": 209}, {"activate": 1, "name": "Parque Santander", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 8, "number": "202", "longitude": "-3.7080833", "no_available": 0, "address": "Calle Santander esq. Avda. Filipinas", "latitude": "40.4411388", "dock_bikes": 16, "id": 210}, {"activate": 1, "name": "San Francisco de Sales", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 15, "number": "203", "longitude": "-3.7144722", "no_available": 0, "address": "General Ampudia, 2", "latitude": "40.44175", "dock_bikes": 8, "id": 211}, {"activate": 1, "name": "Cuatro Caminos 2", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 10, "number": "204", "longitude": "-3.7041666", "no_available": 0, "address": "Doctor Santero 1", "latitude": "40.4476666", "dock_bikes": 13, "id": 212}, {"activate": 1, "name": "Estrecho", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 10, "number": "205", "longitude": "-3.7032777", "no_available": 0, "address": "Navarra, 1", "latitude": "40.4535", "dock_bikes": 10, "id": 213}, {"activate": 1, "name": "Metro Tetu\u00e1n", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 7, "number": "206", "longitude": "-3.6989925", "no_available": 0, "address": "Marqu\u00e9s de Viana, 3", "latitude": "40.4607502", "dock_bikes": 17, "id": 214}, {"activate": 1, "name": "Remonta", "reservations_count": 0, "light": 2, "total_bases": 18, "free_bases": 8, "number": "207", "longitude": "-3.6973333", "no_available": 0, "address": "Azucenas, 1", "latitude": "40.4630277", "dock_bikes": 10, "id": 215}, {"activate": 1, "name": "Plaza de Castilla", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 9, "number": "208", "longitude": "-3.6886388", "no_available": 0, "address": "Paseo Castellana, 216", "latitude": "40.4663611", "dock_bikes": 9, "id": 216}, {"activate": 1, "name": "Plaza de Castilla 2", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 2, "number": "209", "longitude": "-3.6887722", "no_available": 0, "address": "Paseo Castellana (Fundaci\u00f3n Canal)", "latitude": "40.4656779", "dock_bikes": 17, "id": 217}, {"activate": 1, "name": "Tres Cruces", "reservations_count": 0, "light": 0, "total_bases": 27, "free_bases": 27, "number": "210", "longitude": "-3.7026735", "no_available": 0, "address": "Tres Cruces 7", "latitude": "40.419674", "dock_bikes": 0, "id": 218}, {"activate": 1, "name": "Desenga\u00f1o", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 6, "number": "211", "longitude": "-3.70239", "no_available": 0, "address": "Desenga\u00f1o 1", "latitude": "40.42059", "dock_bikes": 17, "id": 219}, {"activate": 1, "name": "Marqu\u00e9s de Vadillo", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 0, "number": "212", "longitude": "-3.716591", "no_available": 0, "address": "Marqu\u00e9s de Vadillo, 9", "latitude": "40.398247", "dock_bikes": 24, "id": 220}, {"activate": 1, "name": "Glorieta de C\u00e1diz", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 5, "number": "213", "longitude": "-3.70017", "no_available": 0, "address": "Antonio L\u00f3pez, 129", "latitude": "40.38895", "dock_bikes": 19, "id": 221}, {"activate": 1, "name": "Condesa de Venadito", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 11, "number": "214", "longitude": "-3.65741", "no_available": 0, "address": "Condesa de Venadito, 2 / Calle Albacete", "latitude": "40.44315", "dock_bikes": 9, "id": 222}, {"activate": 1, "name": "Gutierre de Cetina", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 14, "number": "215", "longitude": "-3.640994", "no_available": 0, "address": "Gutierre de Cetina, 77", "latitude": "40.429706", "dock_bikes": 8, "id": 223}, {"activate": 1, "name": "Puerta del \u00c1ngel", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 0, "number": "216", "longitude": "-3.728318", "no_available": 0, "address": "Plaza Puerta del \u00c1ngel, 4", "latitude": "40.413764", "dock_bikes": 24, "id": 224}, {"activate": 1, "name": "Pedro Rico", "reservations_count": 0, "light": 0, "total_bases": 24, "free_bases": 20, "number": "217", "longitude": "-3.688463", "no_available": 0, "address": "Pedro Rico, 4", "latitude": "40.481086", "dock_bikes": 2, "id": 225}, {"activate": 1, "name": "Camino Vinateros", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 10, "number": "218", "longitude": "-3.65779", "no_available": 0, "address": "Camino Vinateros, 28", "latitude": "40.41053", "dock_bikes": 12, "id": 226}, {"activate": 1, "name": "Marroquina", "reservations_count": 0, "light": 1, "total_bases": 24, "free_bases": 3, "number": "219", "longitude": "-3.646697", "no_available": 0, "address": "Camino Vinateros, 108", "latitude": "40.408264", "dock_bikes": 18, "id": 227}, {"activate": 1, "name": "Plaza del Encuentro", "reservations_count": 1, "light": 2, "total_bases": 24, "free_bases": 10, "number": "220", "longitude": "-3.65122", "no_available": 0, "address": "Calle Arroyo de las Pilillas, 6", "latitude": "40.40617", "dock_bikes": 14, "id": 228}, {"activate": 1, "name": "Pavones", "reservations_count": 0, "light": 2, "total_bases": 24, "free_bases": 15, "number": "221", "longitude": "-3.634587", "no_available": 0, "address": "Hacienda de Pavones, 342", "latitude": "40.400368", "dock_bikes": 8, "id": 229}]}
t.keys()
t['stations']




[{'activate': 1,
  'name': 'Puerta del Sol A',
  'reservations_count': 0,
  'light': 0,
  'total_bases': 30,
  'free_bases': 26,
  'number': '1a',
  'longitude': '-3.7018341',
  'no_available': 0,
  'address': 'Puerta del Sol nº 1',
  'latitude': '40.4172137',
  'dock_bikes': 3,
  'id': 1},
 {'activate': 1,
  'name': 'Puerta del Sol B',
  'reservations_count': 0,
  'light': 2,
  'total_bases': 30,
  'free_bases': 18,
  'number': '1b',
  'longitude': '-3.701602938060457',
  'no_available': 0,
  'address': 'Puerta del Sol nº 1',
  'latitude': '40.41731271011562',
  'dock_bikes': 9,
  'id': 2},
 {'activate': 1,
  'name': 'Miguel Moya',
  'reservations_count': 0,
  'light': 0,
  'total_bases': 24,
  'free_bases': 19,
  'number': '2',
  'longitude': '-3.7058415',
  'no_available': 0,
  'address': 'Calle Miguel Moya nº 1',
  'latitude': '40.4205886',
  'dock_bikes': 3,
  'id': 3},
 {'activate': 1,
  'name': 'Plaza Conde Suchil',
  'reservations_count': 0,
  'light': 1,
  'total_bases': 18,
 

In [ ]:
t[0]

KeyError: 0